## VAR Model Order Selection

**Functions**

`tsa.VAR`

### Exercise 91
Using the same data as in the previous exercise, determine the optimal VAR order using:

1. AIC
2. HQIC
3. BIC
4. Likelihood-ratio testing using General-to-Specific

In [1]:
import pandas as pd

data = pd.read_hdf("./data/var-data.h5", "var_data")

In [2]:
import statsmodels.tsa.api as tsa

mod = tsa.VAR(data[["spread", "gs1", "deflg"]])
res = mod.fit(8, ic="bic")
res.summary()

  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Wed, 27, Aug, 2025
Time:                     15:13:48
--------------------------------------------------------------------
No. of Equations:         3.00000    BIC:                   -15.1574
Nobs:                     264.000    HQIC:                  -15.3276
Log likelihood:           935.526    FPE:                1.96653e-07
AIC:                     -15.4419    Det(Omega_mle):     1.81805e-07
--------------------------------------------------------------------
Results for equation spread
               coefficient       std. error           t-stat            prob
----------------------------------------------------------------------------
const             0.112254         0.072096            1.557           0.119
L1.spread         1.068581         0.098461           10.853           0.000
L1.gs1           -0.051826         0.061315           -0.845           0.39

#### Explanation

statsmodels `VAR` will automatically perform lag length selection using any of the three information criteria. Here we see the BIC chooses 2 lags.  We will implement this directly to make sure we understand what is going on.

In [3]:
import numpy as np
from scipy import stats

ics = pd.DataFrame(
    columns=["AIC", "HQIC", "BIC", "LLF"], index=np.arange(9), dtype=np.double
)
var_data = data[["spread", "gs1", "deflg"]]
for lag in range(9):
    mod = tsa.VAR(var_data.iloc[(8 - lag) :])
    res = mod.fit(lag, ic=None)
    ics.loc[lag, "AIC"] = res.aic
    ics.loc[lag, "HQIC"] = res.hqic
    ics.loc[lag, "BIC"] = res.bic
    ics.loc[lag, "LLF"] = res.llf
ics["LR"] = 2 * (ics.LLF - ics.LLF.shift(1))
# Fill the nan with 0
ics = ics.fillna(0.0)
ics["pval"] = 1 - stats.chi2(9).cdf(ics.LR)
# Pretty the values by truncating at 3 d.p.
ics.round(3)

,AIC,HQIC,BIC,LLF,LR,pval
0,-8.750,-8.733,-8.709,33.499,0.000,1.000
1,-15.183,-15.117,-15.018,872.389,1677.780,0.000
2,-15.378,-15.262,-15.089,906.483,68.188,0.000
3,-15.400,-15.234,-14.987,918.308,23.649,0.005
4,-15.461,-15.245,-14.924,935.255,33.894,0.000
5,-15.436,-15.170,-14.775,941.020,11.530,0.241
6,-15.531,-15.216,-14.746,962.290,42.540,0.000
7,-15.536,-15.171,-14.627,971.927,19.275,0.023
8,-15.578,-15.163,-14.545,986.309,28.763,0.001


#### Explanation

We loop over lag lengths from 0 to 8, estimate the model and save the IC or log - likelihood. Note that we need to adjust the same in each iteration to ensure that the model are comparable since they must be estimated on the same sample data.  This means that we drop 8 observations when there are no lags, 7 when there is 1, and so on until we drop none when there are 8 lags. 

In [4]:
ics[["AIC", "HQIC", "BIC"]].idxmin()

AIC     8
HQIC    2
BIC     2
dtype: int64

#### Explanation

We can use `idxmin` to find the minimum IC in each of the three IC columns.  Both the BIC and HQIC select 2 lags while the AIC likes 8. The LR in row $i$ is for the test that $i$ lags fits the same as $i-1$ lags. For example, the test statistic in row 2 tests whether a model with 2 lags fits as well as a model with 1. The likelihood-ratio testing would select 5 lags if used in with Specific-to-General search, or 8 if used with General-to-Specific.

#### Explanation

We can use `idxmin` to find the minimum IC in each of the three IC columns.  Both the BIC and HQIC select 2 lags while the AIC likes 8. The LR in row $i$ is for the test that $i$ lags fits the same as $i-1$ lags. For example, the test statistic in row 2 tests whether a model with 2 lags fits as well as a model with 1. The likelihood-ratio testing would select 5 lags if used in with Specific-to-General search, or 8 if used with General-to-Specific.

In [5]:
ics[["AIC", "HQIC", "BIC"]].idxmin()

AIC     8
HQIC    2
BIC     2
dtype: int64

#### Explanation

We can use `idxmin` to find the minimum IC in each of the three IC columns.  Both the BIC and HQIC select 2 lags while the AIC likes 8. The LR in row $i$ is for the test that $i$ lags fits the same as $i-1$ lags. For example, the test statistic in row 2 tests whether a model with 2 lags fits as well as a model with 1. The likelihood-ratio testing would select 5 lags if used in with Specific-to-General search, or 8 if used with General-to-Specific.

#### Explanation

We can use `idxmin` to find the minimum IC in each of the three IC columns.  Both the BIC and HQIC select 2 lags while the AIC likes 8. The LR in row $i$ is for the test that $i$ lags fits the same as $i-1$ lags. For example, the test statistic in row 2 tests whether a model with 2 lags fits as well as a model with 1. The likelihood-ratio testing would select 5 lags if used in with Specific-to-General search, or 8 if used with General-to-Specific.

In [6]:
ics[["AIC", "HQIC", "BIC"]].idxmin()

AIC     8
HQIC    2
BIC     2
dtype: int64

#### Explanation

We can use `idxmin` to find the minimum IC in each of the three IC columns.  Both the BIC and HQIC select 2 lags while the AIC likes 8. The LR in row $i$ is for the test that $i$ lags fits the same as $i-1$ lags. For example, the test statistic in row 2 tests whether a model with 2 lags fits as well as a model with 1. The likelihood-ratio testing would select 5 lags if used in with Specific-to-General search, or 8 if used with General-to-Specific.